In [1]:
import django
import os
import sys
import shutil

path='../'

source_file_path = f'{path}.env'

destination_file_path = '.'

shutil.copy(source_file_path, destination_file_path)

project_path = path  # Adjust this to your actual project path
sys.path.append(project_path)
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'oogway.settings')
django.setup()


In [2]:
from PostAnalyzer.models import (
    Channel,
    PositionSide,
    Market,
    PostStatus,
    MarginMode,
    Symbol,
    SettingConfig,
)
from Shared.Exchange import exchange
from Shared.Constant import PostStatusTypeValues, MarketValues, PositionSideValues, MarginModeValues, SETTING_CONFIGS_VARIABLES
from dotenv import dotenv_values

_config = dotenv_values(".env")



In [3]:
async def add_new_channel(channel_id, name, can_trade):
    # Check if already exists in the database
    existing = await Channel.objects.filter(name=name, channel_id=channel_id).aexists()
    if not existing:
        # Create a new Channel instance
        new_channel = Channel(
            channel_id=channel_id,
            name=name,
            can_trade=can_trade
        )
        
        # Save the new channel to the database
        await new_channel.asave()



In [4]:
async def add_post_statuses():
    for status in PostStatusTypeValues:
        for type in status.value[1:]:

            name = status.value[0]
            type_value = int(type)

            # Check if already exists in the database
            # existing = await sync_to_async(PostStatus.objects.filter(name=name, type=type_value).exists)()
            existing = await PostStatus.objects.filter(name=name, type=type_value).aexists()
            
            if not existing:
                # Create if it doesn't exist
                position_side = PostStatus(
                    name=name,
                    type=type_value
                )

                # Save to the database
                await position_side.asave()


In [5]:
async def add_markets():
    for status in MarketValues:
        name = status.value

        # Check if already exists in the database
        existing = await Market.objects.filter(name=name).aexists()
        if not existing:
            # Create if it doesn't exist
            market = Market(
                name=name,
            )

            # Save to the database
            await market.asave()

            



In [6]:
async def add_margin_mode():
    for status in MarginModeValues:
        name = status.value

        # Check if already exists in the database
        existing = await MarginMode.objects.filter(name=name).aexists()
        if not existing:
            # Create if it doesn't exist
            margin = MarginMode(
                name=name,
            )

            # Save to the database
            await margin.asave()

            


In [7]:
async def add_position_size():
    for status in PositionSideValues:
        name = status.value

        # Check if already exists in the database
        existing = await PositionSide.objects.filter(name=name).aexists()
        
        if not existing:
            # Create if it doesn't exist
            position = PositionSide(
                name=name,
            )

            # Save to the database
            await position.asave()

In [8]:
async def add_symbols():
    order_data = exchange.fetch_markets()

    spot_market = await Market.objects.aget(name=MarketValues.SPOT.value)
    futures_market = await Market.objects.aget(name=MarketValues.FUTURES.value)

    for symbol in order_data:
        if symbol["quote"] == "USDT":
            market = spot_market if symbol["spot"] else futures_market

            existing = await Symbol.objects.filter(name=symbol["symbol"], market=market).aexists()

            if not existing:
                newSymbol = {
                    "name": symbol["symbol"],
                    "min_trade_amount": symbol["limits"]["amount"]["min"],
                    "quote": symbol["quote"],
                    "base": symbol["base"],
                    "market": market,
                }

                newSymbol = Symbol(**newSymbol)
                await newSymbol.asave()

In [9]:
async def add_settingConfig():
    
    existing = await SettingConfig.objects.aexists()

    if not existing:

        newSymbol = SettingConfig(**SETTING_CONFIGS_VARIABLES)
        await newSymbol.asave()



In [10]:
await add_margin_mode()
await add_markets()
await add_post_statuses()
await add_position_size()
await add_symbols()
await add_settingConfig()


# add channel info here
channels = [
    {
        'name': "TEST_FEYZIAN",
        'id': _config["CHANNEL_TEST_FEYZIAN"],
    },
    {
        'name': "FEYZIAN",
        'id': _config["CHANNEL_FEYZ"],
    },
    {
        'name': "ALI_BEY",
        'id': _config["CHANNEL_ALI_BEY"],
    },
    {
        'name': "TEST_ALI_BEYRANVAND",
        'id': _config["CHANNEL_TEST_ALI_BEYRANVAND"],
    }, 
    {
        'name': "RASTAD",
        'id': _config["CHANNEL_RASTAD"],
    },
    {
        'name': "TEST_RASTAD",
        'id': _config["CHANNEL_TEST_RASTAD"],
    }, 
    {
        'name': "SAMAN_SADEGHI",
        'id': _config["CHANNEL_SAMAN_SADEGHI"],
    }, 
    {
        'name': "KING_CRYPTO",
        'id': _config["CHANNEL_KING_CRYPTO"],
    }, 
    {
        'name': "ROSE",
        'id': _config["CHANNEL_ROSE"],
    },

]

for item in channels:
    await add_new_channel(item['id'], item['name'], False)